# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
X = data.iloc[:, 3:-1].values
y = data.iloc[:, -1].values

In [3]:
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


### Encoding categorical data

Label Encoding the "Gender" column

In [4]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

In [5]:
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


In [6]:
print(X[:, 2][5]) # value for 'male'

1


One Hot Encoding the "Geography" column

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [8]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


In [9]:
print(X[:, 0][0]) # value for 'France'

1.0


### Splitting the dataset into the Training set and Test set

In [10]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

## Part 2 - Building the ANN

### Initializing the ANN

In [12]:
ann = tf.keras.models.Sequential()


### Adding the input layer and the first hidden layer

In [13]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

### Adding the second hidden layer

In [14]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

### Adding the third hidden layer

In [15]:
ann.add(tf.keras.layers.Dense(units=10, activation='relu'))

### Adding the output layer

In [16]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Part 3 - Training the ANN

### Compiling the ANN

In [17]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the Training set

In [18]:
ann.fit(X_train, y_train, batch_size=32, epochs=100 )

Epoch 1/100
250/250 [==============================] - 2s 2ms/step - loss: 0.4877 - accuracy: 0.7960
Epoch 2/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4454 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4265 - accuracy: 0.7979
Epoch 4/100
250/250 [==============================] - 1s 2ms/step - loss: 0.4097 - accuracy: 0.8194
Epoch 5/100
250/250 [==============================] - 1s 3ms/step - loss: 0.3889 - accuracy: 0.8346
Epoch 6/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3702 - accuracy: 0.8468
Epoch 7/100
250/250 [==============================] - 1s 4ms/step - loss: 0.3596 - accuracy: 0.8516
Epoch 8/100
250/250 [==============================] - 1s 5ms/step - loss: 0.3551 - accuracy: 0.8535
Epoch 9/100
250/250 [==============================] - 2s 7ms/step - loss: 0.3514 - accuracy: 0.8553
Epoch 10/100
250/250 [==============================] - 2s 6ms/step - loss: 0.3493 - accura

## Part 4 - Making the predictions and evaluating the model

### Predicting the result of a single observation

**Homework**

Use our ANN model to predict if the customer with the following informations will leave the bank: 

Geography: France

Credit Score: 600

Gender: Male

Age: 40 years old

Tenure: 3 years

Balance: \$ 60000

Number of Products: 2

Does this customer have a credit card? Yes

Is this customer an Active Member: Yes

Estimated Salary: \$ 50000

So, should we say goodbye to that customer?

**Solution**

In [19]:
customer_stays = ann.predict(
    sc.transform(
        [
            [1, 0, 0, 600, 1, 40, 3, 60000, 2, 1, 1, 50000]
        ]
    )
) > 0.5

print(customer_stays)

1/1 [==============================] - 0s 132ms/step
[[False]]


### Predicting the Test set results

In [20]:
y_pred = ann.predict(X_test) > 0.5
print(np.concatenate(
    (
        y_pred.reshape(len(y_pred), 1),
        y_test.reshape(len(y_test), 1)
    ), 1
))

63/63 [==============================] - 0s 2ms/step
[[0 0]
 [0 1]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 0]]


### Making the Confusion Matrix

In [21]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_pred, y_test)
acs = accuracy_score(y_pred, y_test)

print(cm)
print(acs)

[[1535  211]
 [  60  194]]
0.8645
